In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Trade To Portfolio Rate (TTPR) Demo

This notebook demonstrates how LUSID can resolve the Trade To Portfolio Rate for transactions booked with different trade currencies to the base portfolio currency.

Attributes
----------
portfolios
instruments
quotes
recipes
transactions
trade to portfolio rate
"""

toggle_code("Hide docstring")

# Trade To Portfolio Rate (TTPR) Example Notebook

*** DISCLAIMER ***

In order to run this notebook, code in sections 1.1 and 7. needs to be uncommented.
Be aware that this will have system wide implications for booking transactions (beyond the scope of this notebook) as enabling the "SetTradeToPortfolioRate" field enables it across the entire LUSID domain.

This notebook demonstrates how LUSID can resolve the Trade To Portfolio Rate for transactions booked with different trade currencies to the base portfolio currency.

In this example, 5 transactions in 5 different currencies are booked to a GBP portfolio and the associated TTPRs returned.

## 1. Setup LUSID

In [2]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
import lusid_configuration
from lusid_configuration import ConfigurationSetsApi
from lusid_configuration import models as config_models
from fbnsdkutilities import ApiClientFactory as ConfigApiClientFactory

from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd
import json
import pytz
from datetime import datetime

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
configuration_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "configuration"

config_api_factory = ConfigApiClientFactory(
    lusid_configuration,
    token=api_client.configuration.access_token,
    api_url=configuration_api_url,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Define the apis
quotes_api = lusid_api_factory.build(lusid.QuotesApi)
configuration_recipe_api = lusid_api_factory.build(lusid.api.ConfigurationRecipeApi)
instruments_api = lusid_api_factory.build(lusid.InstrumentsApi)
transaction_portfolios_api = lusid_api_factory.build(lusid.TransactionPortfoliosApi)
sets_api = config_api_factory.build(lusid_configuration.api.ConfigurationSetsApi)

## 1.1  Set Up Configuration Store

Set the "SetTradeToPortfolioRate" item to True to enable the calculation of the trade to portfolio rate in valuations.
<span style="color:red">(This will need to be uncommented)</span> 

(Details on TradeToPortfolioRate property can be found here - https://support.lusid.com/knowledgebase/article/KA-01744/en-us#transaction)

Also, set the scope and code of the "TradeToPortfolioRateRecipe" item (RecipeScope/RecipeCode)

In [4]:
###########
# WARNING #
###########

# Turning on this property is a system wide change and could effect other processes.

# config_ttpr_response = sets_api.update_configuration_item(
#     type="Shared",
#     scope="system",
#     code="TransactionBooking",
#     key="SetTradeToPortfolioRate",
#     update_configuration_item = config_models.UpdateConfigurationItem(value="True")
# )

config_ttpr_recipe_response = sets_api.update_configuration_item(
    type="Shared",
    scope="system",
    code="TransactionBooking",
    key="TradeToPortfolioRateRecipe",
    update_configuration_item = config_models.UpdateConfigurationItem(value="IBOR/StandardMarketValue")
)

In [5]:
# Define variables
valuation_date = datetime(year=2022, month=3, day=8, tzinfo=pytz.UTC)
settlement_date = datetime(year=2022, month=3, day=12, tzinfo=pytz.UTC)
portfolio_scope = "TTPR_NB"
portfolio_code = "TTPR_NB"

## 2. Create Portfolio

Create a new portfolio with base currency GBP.

In [6]:
try: 
    transaction_portfolios_api.create_portfolio(
        scope=portfolio_scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name="TTPR_NB",
            code=portfolio_code,
            created="2020-01-01T00:00:00+00:00",
            base_currency="GBP",
            instrument_scopes=[portfolio_scope]
        )
    )
    
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'TTPR_NB' because it already exists in scope 'TTPR_NB'.


## 3. Load Instruments

Load 5 instruments into the portfolio.

In [7]:
# Define 5 example instruments

for i in range(1, 6):
    instrument_body = models.InstrumentDefinition(
        name=f"instrument_{i}",
        identifiers={"ClientInternal": models.InstrumentIdValue(value=f"client_internal_{i}")}
    )
    
    reponse = instruments_api.upsert_instruments(
        scope=portfolio_scope,
        request_body={
            "request_id_1": instrument_body
        }
    )

## 4. Load Quotes

### 4.1 FX Rate Quotes

Load 5 different FX rate quotes for the 5 different currency transactions booked into this portfolio (see below).

In [8]:
# FX rate quote upsertion function
def spot_request(from_ccy, to_ccy, rate, valuation_date):
            return models.UpsertQuoteRequest(
                       quote_id=models.QuoteId(
                           models.QuoteSeriesId(
                               provider='Lusid',
                               instrument_id=f'{from_ccy}/{to_ccy}',
                               instrument_id_type='CurrencyPair',
                               quote_type='Rate',
                               field='mid'
                           ),
                           effective_at=valuation_date
                       ),
                       metric_value=models.MetricValue(
                           value=rate,
                           unit=f'{from_ccy}/{to_ccy}'
                       ),
                       lineage='None'
            )

In [9]:
fx_rates = [["USD", 1.3106], ["AUD", 1.7909], ["JPY", 151.14], ["EUR", 1.2073], ["CAD", 1.6803]]

for rate in fx_rates:
    response = quotes_api.upsert_quotes(scope=portfolio_scope,
                                   request_body={"1": spot_request("GBP", rate[0], rate[1], valuation_date)})

## 5. Define Default Recipe

Define the default valuation recipe to be linked to the trade to portfolio rate configuration in the configuration store.

In [10]:
# Create recipes
recipe_scope="IBOR"
recipe_code="TTPR_NB_Recipe"


# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            models.MarketDataKeyRule(
                key='Fx.CurrencyPair.*',
                data_scope=portfolio_scope,
                supplier='Lusid',
                quote_type='Rate',
                quote_interval='1D.0D',
                field="mid"
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope='Lusid',
            # This enables FX rate inference
            attempt_to_infer_missing_fx=True,

        ),
    ),
    pricing=models.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": True},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## 6. Load Transactions

Load 5 transactions into LUSID in 5 different currencies.

In [11]:
# Upsert example transaction data for 5 instruments
cost = [15.25, 25.70, 57.34, 100.99, 23.19]
currency = ["USD", "AUD", "JPY", "EUR", "CAD"]

for i in range(1,6):
    transaction_portfolios_api.upsert_transactions(
        scope=portfolio_scope,
        code=portfolio_code,
        transaction_request=[models.TransactionRequest(
            transaction_id=f"transaction_id_{i}",
            type="Buy",
            instrument_identifiers={
                "instrument/default/ClientInternal" : f"client_internal_{i}"
            },
            transaction_date=valuation_date,
            settlement_date=settlement_date,
            units=100,
            total_consideration = models.CurrencyAndAmount(
                amount=cost[i-1],
                currency=currency[i-1],
            )
        )]
    )

## 7. Get Transactions

Get the transactions data including the trade to portfolio rates calculated in LUSID from the upserted fx rate quotes.

<span style="color:red">(Uncomment lines 24 and 35 once the Trade To Portfolio Rate option has been set to true in the configuration store.)</span>

In [13]:
get_holdings_reponse = transaction_portfolios_api.get_holdings(
    scope = portfolio_scope,
    code = portfolio_code,
    effective_at = valuation_date
)

get_transactions_reponse = transaction_portfolios_api.get_transactions(
    scope = portfolio_scope,
    code = portfolio_code,
)

holdings_df_all = lusid_response_to_data_frame(get_holdings_reponse)
transactions_df_all = lusid_response_to_data_frame(get_transactions_reponse)
combined_data_df_all = transactions_df_all.merge(holdings_df_all, on='instrument_uid', how='left')

# Column selection
combined_data_df = combined_data_df_all[[
    "transaction_id", 
    "type", 
    "instrument_identifiers.Instrument/default/ClientInternal",
    "instrument_uid",
    "transaction_currency",
    "cost_portfolio_ccy.currency",
#     "properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value"
]]

# Column renaming
combined_data_df = combined_data_df.rename(
    columns={
        "transaction_id" : "transactionId", 
        "instrument_identifiers.Instrument/default/ClientInternal" : "ClientInternal",
        "instrument_uid" : "LUID",
        "transaction_currency" : "TradeCurrency",
        "cost_portfolio_ccy.currency" : "PortfolioCurrency",
#         "properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value" : "TTPR"
    }
)

combined_data_df

,transactionId,type,ClientInternal,LUID,TradeCurrency,PortfolioCurrency
0,transaction_id_1,Buy,client_internal_1,LUID_00003F35,USD,GBP
1,transaction_id_2,Buy,client_internal_2,LUID_00003F36,AUD,GBP
2,transaction_id_3,Buy,client_internal_3,LUID_00003F37,JPY,GBP
3,transaction_id_4,Buy,client_internal_4,LUID_00003F38,EUR,GBP
4,transaction_id_5,Buy,client_internal_5,LUID_00003F39,CAD,GBP
